# Gym Exercise Repetition Counter and Classifier
## Introducción
El objetivo de este proyecto es desarrollar un modelo de Machine Learning que permita contar repeticiones y predecir el tipo de ejercicio realizado en el gimnasio en tiempo real. Los datos provienen de sensores de acelerómetro y giroscopio integrados en un smartwatch. Esto busca mejorar la experiencia del usuario, optimizar el entrenamiento y reducir el riesgo de lesiones.


## 1. Preparación de datos

### Importación de bibliotecas necesarias

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import LocalOutlierFactor
from scipy.signal import butter, lfilter, filtfilt
from scipy.special import erf
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

### Carga y exploración inicial de los datos

In [ ]:
# Cargar el dataset en formato Pickle
raw_data = pd.read_pickle("src/data/01_Data_Processed.pkl")
print("Primeras filas del dataset:")
print(raw_data.head())
# Información general del dataset
raw_data.info()

## 2. Detección y tratamiento de outliers
### Métodos de detección de outliers

In [ ]:
def mark_outliers_iqr(dataset, col):
    Q1 = dataset[col].quantile(0.25)
    Q3 = dataset[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    dataset[col + "_outlier"] = (dataset[col] < lower_bound) | (dataset[col] > upper_bound)
    return dataset

def mark_outliers_chauvenet(dataset, col, C=2):
    mean = dataset[col].mean()
    std = dataset[col].std()
    N = len(dataset.index)
    criterion = 1.0 / (C * N)

    deviation = abs(dataset[col] - mean) / std

    prob = 1.0 - 0.5 * (erf(deviation / np.sqrt(2)) - erf(-deviation / np.sqrt(2)))
    dataset[col + "_outlier"] = prob < criterion
    return dataset

### Filtrado de outliers

In [ ]:
outlier_columns = raw_data.columns[1:7]
outliers_removed = raw_data.copy()

for col in outlier_columns:
    for label in raw_data["Label"].unique():
        subset = mark_outliers_chauvenet(raw_data[raw_data["Label"] == label], col)
        subset.loc[subset[col + "_outlier"], col] = np.nan
        outliers_removed.update(subset)

# Guardar el dataset sin outliers
outliers_removed.to_pickle("../data/02_outliers_removed.pkl")

## 3. Generación de características (Feature Engineering)

### Transformaciones temporales y de frecuencia

In [ ]:
# Implementación de la transformación de Fourier y abstracciones temporales
def fourier_transformation(data, col, window_size, sampling_rate):
    freqs = np.fft.rfftfreq(window_size, 1 / sampling_rate)
    fft_values = np.fft.rfft(data[col])
    return freqs, abs(fft_values)

def temporal_abstraction(data, cols, window_size, aggregation):
    for col in cols:
        data[col + f"_{aggregation}_ws{window_size}"] = data[col].rolling(window_size).aggregate(aggregation)
    return data

### Generación de características combinadas

In [ ]:
# Crear características adicionales
processed_data = pd.read_pickle("../data/02_outliers_removed.pkl")
processed_data["acc_r"] = np.sqrt(
    processed_data["Accelerometer_x"] ** 2 + 
    processed_data["Accelerometer_y"] ** 2 + 
    processed_data["Accelerometer_z"] ** 2
)

# Aplicar abstracciones temporales
processed_data = temporal_abstraction(processed_data, ["acc_r"], window_size=5, aggregation="mean")

# Guardar el dataset con nuevas características
processed_data.to_pickle("../data/03_data_features.pkl")

## 4. Entrenamiento y evaluación de modelos

### División de datos

In [ ]:
data = pd.read_pickle("../data/03_data_features.pkl")
X = data.drop(["Label", "Participants", "Set"], axis=1)
y = data["Label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

### Entrenamiento de modelos

In [ ]:
# Inicializar clasificadores
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Red Neuronal": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500),
    "SVM": SVC(kernel="rbf", probability=True),
    "Naive Bayes": GaussianNB(),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

# Entrenar y evaluar
results = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[model_name] = accuracy
    print(f"{model_name} Accuracy: {accuracy:.4f}")

### Comparación de resultados

In [ ]:
# Visualizar resultados
plt.figure(figsize=(10, 5))
plt.bar(results.keys(), results.values())
plt.title("Comparación de Precisión entre Modelos")
plt.ylabel("Precisión")
plt.xticks(rotation=45)
plt.show()

## 5. Conclusión

1. **Resumen**: Evaluamos múltiples modelos de clasificación y obtuvimos el mejor rendimiento con `Random Forest`.
2. **Aplicaciones**: Este sistema permite contar repeticiones y clasificar ejercicios en tiempo real, optimizando entrenamientos.
3. **Trabajo futuro**: Integrar el modelo en una aplicación móvil o smartwatch para uso en tiempo real.
